# Central Park Centrality
While I lived in NYC I remember whenever I'd walk through Central Park I'd find I'd always end up at Bethesda Fountain at some point during my walk.  
The famous PageRank represents the odds of landing of a given webpage, after randomly clicking links and a given probably factor of stay at the current page.  
This makes me wonder, what happens if we run PageRank and other graph centrality algorithms on the map of paths in Central park? Where does Bethesda fountain rank, and what are the actual central nodes in Central Park?

In [ ]:
# !pip install -r https://raw.githubusercontent.com/gboeing/osmnx/main/requirements.txt
# !pip install osmnx

## Imports

In [ ]:
# Graph toolset
import networkx as nx  # For graph operations
import osmnx as ox  # For downloading and manipulating OpenStreetMap data in NetworkX format

import pandas as pd  # For manipulting tables of data
import geopandas as gpd  # For spacial operations on tables of data

import numpy as np  # For arrays and matrix operations

import scipy.sparse  # For sparse arrays
import scipy.interpolate

In [ ]:
# Visualization toolset
import matplotlib.pyplot as plt
from matplotlib import colormaps
from matplotlib.animation import FuncAnimation

from IPython.display import HTML

%matplotlib inline
# %matplotlib notebook
#plt.rcParams['figure.figsize'] = [8, 8]

## Map Data

### Central Park
First things first, let's download the graph of Central Park, which we can do by name with the OSMnx API.

In [ ]:
G = ox.graph.graph_from_place("Central Park", network_type='walk', truncate_by_edge=True)
gdf,egdf = ox.utils_graph.graph_to_gdfs(G)
fig, ax = ox.plot_graph(G, node_size=1)

### Bethesda Fountain
We want all nodes of our graph associated with Bethesda Fountain, so let's again use OSMnx's API to find them by name.

In [ ]:
target_gdf = ox.geometries.geometries_from_place("Bethesda Fountain", {'name':'Bethesda Fountain'})
for ktype,osmid in target_gdf.index:
    print(f"https://www.openstreetmap.org/{ktype}/{osmid}")
target_gdf

Note that since we get a 'way' element_type, we need to find the nodes in the Central Park graph which are closest to point in the Bethesda Fountain way, which can be done as follows:

In [ ]:
polygon = target_gdf.loc['way']['geometry'].iloc[0]
target_nodes = list(set(ox.distance.nearest_nodes(G, *zip(*polygon.exterior.coords))))
for node in target_nodes:
    print(f"https://www.openstreetmap.org/node/{node}")

In [ ]:
fig, ax = ox.plot_graph(G, show=False, close=False, node_size=1)
ax.scatter(gdf['x'][target_nodes], gdf['y'][target_nodes], color='yellow', s=75, edgecolors='black')
plt.show()

### Columbus Circle
Generally I would start my walks from Columbus Circle, so let's fetch that for a starting point, similar to what we did for Bethesda Fountain.

In [ ]:
start_gdf = ox.geometries.geometries_from_place("Columbus Circle", {'name':'Columbus Circle'})
for ktype,osmid in start_gdf.index:
    print(f"https://www.openstreetmap.org/{ktype}/{osmid}")
start_gdf

Columbus Circle isn't actually a part of Central Park itself, so to find the park entrance from Columbus Circle we can get the nearest node to it.

In [ ]:
point = start_gdf.loc['node']['geometry'].iloc[0]
start_node = ox.distance.nearest_nodes(G, *zip(*point.coords))[0]
start_node_index = gdf.index.get_loc(start_node)
print(f"https://www.openstreetmap.org/node/{start_node}")

In [ ]:
fig, ax = ox.plot_graph(G, show=False, close=False, node_size=1)
ax.scatter(gdf['x'][start_node], gdf['y'][start_node], color='yellow', s=75, edgecolors='black')
plt.show()

### Removing Dead-Ends
Dead-ends can trap a random-walker, but in practice I would just walk across the grass, or avoid an obvious dead end.
So let's prune off dead-ends.
We can do this in terms of graph connectivity: a subgraph connected to the rest by a solitary edge is a "dead end".

In [ ]:
# optionally run this cell to remove dead-ends
Gcut = ox.utils_graph.get_undirected(G)
while len(e := nx.minimum_edge_cut(Gcut)) == 1:
    e = e.pop()
    Gcut.remove_edge(*e)
    for node in e:
        if not nx.has_path(Gcut, start_node, node):
            Gcut.remove_nodes_from(nx.descendants(Gcut, node))
            Gcut.remove_node(node)
G = nx.MultiDiGraph(Gcut)
gdf,egdf = ox.utils_graph.graph_to_gdfs(G)
start_node_index = gdf.index.get_loc(start_node)

## Centralities
NetworkX comes with a variety of centrality measures out-of-the-box. Let's take a look at them.

In particular we can see how our target nodes rank up, and graph the edges by mean of the centralities of their head and tail nodes. We can also show the top $n$ nodes.

In [ ]:
def analyze_centrality_measure(G, name, func, show_target_nodes=None, show_top_n=3, plot_interpolated=False):
    gdf,egdf = ox.utils_graph.graph_to_gdfs(G)
    # Compute the centralities
    centralities = func(G)
    centralities_series = pd.Series(centralities)
    nx.set_node_attributes(G, centralities, name)
    
    # Report on the target_nodes
    if show_target_nodes:
        target_ranks = centralities_series.argsort()[target_nodes] 
        target_percentile_ranks = target_ranks / centralities_series.size
        print(f"Target node percentile ranks by {name}")
        print(target_percentile_ranks)
        print()
    
    # plot the interpolated image
    ax = None
    if plot_interpolated:
        xrange = gdf['x'].max()-gdf['x'].min()
        yrange = gdf['y'].max()-gdf['y'].min()
        ex = 0.1
        bounds = [
            gdf['x'].min()-ex*xrange, gdf['x'].max()+ex*xrange,
            gdf['y'].min()-ex*yrange, gdf['y'].max()+ex*yrange,
        ]
        grid_x, grid_y = np.mgrid[bounds[0]:bounds[1]:1000j, bounds[2]:bounds[3]:1000j]
        grid = scipy.interpolate.griddata(gdf[['x','y']].to_numpy(), centralities_series.to_numpy(), (grid_x, grid_y), method='linear', fill_value=0.)
        ax = plt.imshow(grid.T, extent=bounds, origin='lower', cmap='gnuplot').ax

    # plot the network
    edge_mean_centralities = {(u,v,k):(centralities_series[u]+centralities_series[v])/2 for u,v,k in egdf.index}
    nx.set_edge_attributes(G, edge_mean_centralities, name)
    ec = ox.plot.get_edge_colors_by_attr(G, name, cmap="inferno")
    c = ox.plot.get_node_colors_by_attr(G, name, cmap="inferno")
    fig, ax = ox.plot_graph(G, ax=ax, show=False, close=False, edge_color=ec, node_color=c, node_size=5)

    # Show the top 3 nodes
    print(f"Top {show_top_n} nodes by {name}")
    nodes = centralities_series.sort_values(ascending=False).index[:show_top_n]
    for i,node in enumerate(nodes):
        print(f"{i+1}: https://www.openstreetmap.org/node/{node}")
    ax.scatter(gdf['x'][nodes], gdf['y'][nodes], s=75, c=range(show_top_n), cmap='Wistia', edgecolors='black')

    plt.show()

### PageRank
Represents the probability of remaining at a given node, after randomly taking paths, starting from a random node, and stopping after a random amount of time.

In [ ]:
name = "pagerank"
func = nx.pagerank
analyze_centrality_measure(G, name, func, show_target_nodes=target_nodes)

### Closeness Centrality

In [ ]:
name = "closeness_centrality"
func = nx.closeness_centrality
analyze_centrality_measure(G, name, func, show_target_nodes=target_nodes)

### Betweenness Centrality

In [ ]:
name = "betweenness_centrality"
func = nx.betweenness_centrality
analyze_centrality_measure(G, name, func, show_target_nodes=target_nodes)

### Current Flow Closeness Centrality

In [ ]:
name = "current_flow_closeness_centrality"
func = lambda G: nx.current_flow_closeness_centrality(ox.utils_graph.get_undirected(G))
analyze_centrality_measure(G, name, func, show_target_nodes=target_nodes)

### Current Flow Betweenness Centrality

In [ ]:
name = "current_flow_betweenness_centrality"
func = lambda G: nx.current_flow_betweenness_centrality(ox.utils_graph.get_undirected(G))
analyze_centrality_measure(G, name, func, show_target_nodes=target_nodes)

## Random Walk Simulations

In [ ]:
def draw_walk(G, T, steps, start_node=None, name='animation', fps=60):
    gdf,egdf = ox.utils_graph.graph_to_gdfs(G)

    fig, ax = plt.subplots(facecolor="black")
    ax.set_facecolor('black')

    pos = np.ones(T.shape[0]) / T.shape[0]
    if start_node is not None:
        pos[:] = 0
        pos[gdf.index.get_loc(start_node)] = 1

    def draw_function(nframe):
        # update the data
        pos[:] = T.dot(pos)
        # draw the graph
        ax.clear()
        # setup graph data
        pos_series = pd.Series(data=pos, index=gdf.index)
        nx.set_node_attributes(G, pos_series, "animation")
        edge_mean_pos = {(u,v,k):(pos_series[u]+pos_series[v])/2 for u,v,k in egdf.index}
        nx.set_edge_attributes(G, edge_mean_pos, "animation")
        #plot
        ec = ox.plot.get_edge_colors_by_attr(G, "animation", cmap="inferno")
        c = ox.plot.get_node_colors_by_attr(G, "animation", cmap="inferno")
        ox.plot_graph(G, ax=ax, show=False, close=False, bgcolor='black', edge_color=ec, node_color=c, node_size=5)

    anim = FuncAnimation(fig, draw_function, frames=steps, repeat=True)
    #anim.save(f"{name}.gif", fps=fps)
    plt.close()
    return HTML(anim.to_html5_video())

In [ ]:
def analyze_probability_flow(G, T, steps, target_nodes, start_node=None):
    gdf,egdf = ox.utils_graph.graph_to_gdfs(G)
    target_nodes_indices = [gdf.index.get_loc(node) for node in target_nodes]
    # initialize position
    pos = np.ones(T.shape[0]) / T.shape[0]
    if start_node is not None:
        pos[:] = 0
        pos[gdf.index.get_loc(start_node)] = 1
    P = [pos[target_nodes_indices].sum()]
    Pcum = [P[0]]
    for i in range(steps):
        pos[:] = T.dot(pos)
        P.append(pos[target_nodes_indices].sum())
        Pcum.append(Pcum[-1] + (1 - Pcum[-1]) * P[-1])
    # plot
    fig, axs = plt.subplots(2)
    #fig.title('Vertically stacked subplots')
    axs[0].plot(P)
    axs[0].set_title("Probability of being at target node")
    axs[0].set_xlabel("steps")
    axs[0].set_ylabel("probability")
    axs[1].plot(Pcum)
    axs[1].set_title("Cumulative probability of having visited target node")
    axs[1].set_xlabel("steps")
    axs[1].set_ylabel("cumulative probability")
    plt.tight_layout()

### Uniform Adjacency

In [ ]:
A = nx.adjacency_matrix(G)
# normalize to stochastic transition matrix
edge_count = A.sum(axis=1)
invD = scipy.sparse.diags(1/edge_count)
T = A.dot(invD)

In [ ]:
draw_walk(G, T, 60*10, start_node=start_node)

In [ ]:
analyze_probability_flow(G, T, 60*10, target_nodes, start_node=start_node)

### Directionally Weighted Random Walk

In [ ]:
outward_bearings = ox.bearing.calculate_bearing(gdf.loc[start_node]['y'], gdf.loc[start_node]['x'], gdf['y'], gdf['x'])

In [ ]:
def normalized_inner_product_weight(head, tail):
    bearing = ox.bearing.calculate_bearing(gdf.loc[head]['y'], gdf.loc[head]['x'], gdf.loc[tail]['y'], gdf.loc[tail]['x'])
    return 1+np.cos(np.deg2rad(bearing - outward_bearings[head]))/2+0.5

In [ ]:
weight_function = normalized_inner_product_weight

In [ ]:
W = scipy.sparse.dok_matrix(A.shape, dtype='float64')
for i,node in enumerate(G.nodes()):
    for neighbor in G.neighbors(node):
        j = gdf.index.get_loc(neighbor)
        W[i,j] = weight_function(node, neighbor)
W = scipy.sparse.csr_array(W)
# normalize to stochastic transition matrix
weight_sums = W.sum(axis=1)
invD = scipy.sparse.diags(1/weight_sums)
T = invD.dot(W).T

In [ ]:
draw_walk(G, T, 60*10, start_node=start_node)

In [ ]:
analyze_probability_flow(G, T, 60*10, target_nodes, start_node=start_node)

## References
- https://osmnx.readthedocs.io/en/stable/index.html
- https://networkx.org/documentation/stable/reference/algorithms/centrality.html
- https://networkx.org/documentation/stable/reference/algorithms/connectivity.html
- https://github.com/gboeing/osmnx-examples/blob/main/notebooks/00-osmnx-features-demo.ipynb
- https://www.sci.unich.it/~francesc/teaching/network/flowcentrality.html
- https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.Animation.html
- https://en.wikipedia.org/wiki/Connectivity_(graph_theory)
- https://stackoverflow.com/questions/37311651/get-node-list-from-random-walk-in-networkx
- https://stackoverflow.com/questions/43646550/how-to-use-an-update-function-to-animate-a-networkx-graph-in-matplotlib-2-0-0
- https://stackoverflow.com/questions/31815454/animate-graph-diffusion-with-networkx